In [1]:
# import required py modules
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype

In [2]:
# Define Global Quarantine Pandas Categorical Type
# https://rappler.com/newsbreak/iq/explainer-movement-areas-under-community-quarantine-coronavirus
quarantine_types = ['NONE', 'MGCQ', 'GCQ', 'MECQ', 'ECQ']
quarantine_cat_type = CategoricalDtype(categories=quarantine_types, ordered=True)

In [3]:
# Define region demography dataframe
df_reg_demography = pd.read_csv('datasets/Region Demography.csv', dtype={'Location': 'object', 'Region': 'category'})
print(f"Region Demography Data Loaded with Shape {df_reg_demography.shape}")

Region Demography Data Loaded with Shape (44, 2)


In [4]:
# Define declaration timeline dataframe
df_q_dec_timeline = pd.read_csv('datasets/Declaration Timeline.csv', dtype={'Date': 'string', 'isStart': 'bool', 'isEnd': 'bool'})
df_q_dec_timeline['Date'] = pd.to_datetime(df_q_dec_timeline['Date'])
print(f"Declaration Timeline Data Loaded with Shape {df_q_dec_timeline.shape}")

Declaration Timeline Data Loaded with Shape (106, 3)


In [5]:
# Define province extractor function
def province_to_locations_array(region):
    '''
    Returns an array of locations given a region
    Throws KeyError
    
    parameter: region
    returns array
    '''
    return df_reg_demography.query(f"Region == '{region}'")['Location'].values;

def all_provinces_array():
    '''
    Returns an array of all locations
    
    parameter: region
    returns array
    '''
    provinces_3 = province_to_locations_array("III").tolist()
    provinces_4a = province_to_locations_array("IV-A").tolist()
    provinces_ncr = province_to_locations_array("NCR").tolist()
    provinces_9 = province_to_locations_array("VII").tolist()
    provinces_11 = province_to_locations_array("XI").tolist()
    return provinces_3 + provinces_4a + provinces_ncr + provinces_9 + provinces_11

In [6]:
# Define function that will read Quarantine Timeline CSVs
def read_quarantine_timeline(region):
    dtypes = {"Date": 'string'}
    provinces = province_to_locations_array(region)
    for province in provinces:
        dtypes[province] = quarantine_cat_type
    df = pd.read_csv(f"datasets/Quarantine Timeline - {region}.csv", dtype=dtypes)
    df['Date'] = pd.to_datetime(df['Date'])
    return df

In [7]:
# Extract csvs per region
df_quarantine_timeline = {
    "III": read_quarantine_timeline("III"),
    "IV-A": read_quarantine_timeline("IV-A"),
    "NCR": read_quarantine_timeline("NCR"),
    "VII": read_quarantine_timeline("VII"),
    "XI": read_quarantine_timeline("XI")
}

In [8]:
# Join regional data and perform wrangling
df_regional_quarantine_timeline = df_q_dec_timeline
del df_q_dec_timeline
for region in df_quarantine_timeline.keys():
    df_regional_quarantine_timeline = df_regional_quarantine_timeline.merge(df_quarantine_timeline[region], on="Date")
del df_quarantine_timeline
print(f"Data Merged with columns {df_regional_quarantine_timeline.columns}")
print(f"Data is from {df_regional_quarantine_timeline['Date'].min()} to {df_regional_quarantine_timeline['Date'].max()}")

Data Merged with columns Index(['Date', 'isStart', 'isEnd', 'Aurora', 'Bataan', 'Bulacan',
       'Nueva Ecija', 'Pampanga', 'Tarlac', 'Zambales', 'Angeles', 'Olongapo',
       'Batangas', 'Cavite', 'Laguna', 'Rizal', 'Quezon', 'Caloocan',
       'Las Pinas', 'Makati', 'Malabon', 'Mandaluyong', 'Manila', 'Marikina',
       'Muntinlupa', 'Navotas', 'Paranaque', 'Pasay', 'Pasig', 'Pateros',
       'Quezon City', 'San Juan', 'Taguig', 'Valenzuela', 'Bohol', 'Cebu',
       'Negros Oriental', 'Siquijor', 'Cebu City', 'Lapu-lapu City',
       'Mandaue City', 'Davao del Norte', 'Davao del Sur', 'Davao Oriental',
       'Davao Occidental', 'Davao de Oro', 'Davao City'],
      dtype='object')
Data is from 2020-03-17 00:00:00 to 2020-06-30 00:00:00


In [9]:
# Perform Feature Engineering

# Add date details
df_regional_quarantine_timeline['day_of_week'] = df_regional_quarantine_timeline['Date'].dt.dayofweek.astype('uint8')
df_regional_quarantine_timeline['month'] = df_regional_quarantine_timeline['Date'].dt.month.astype('uint8')
df_regional_quarantine_timeline['day_of_month'] = df_regional_quarantine_timeline['Date'].dt.day.astype('uint8')
df_regional_quarantine_timeline['day_of_year'] = df_regional_quarantine_timeline['Date'].dt.dayofyear.astype('uint16')
df_regional_quarantine_timeline['day_name'] = pd.Categorical(
    df_regional_quarantine_timeline['Date'].dt.day_name(),
    categories=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
    ordered=True)

# Add Category Code
provinces = all_provinces_array()
for province in provinces:
    df_regional_quarantine_timeline[f'{province} Q Code'] = df_regional_quarantine_timeline[province].cat.codes.astype('uint8')
del provinces
    
print(df_regional_quarantine_timeline.dtypes)

Date                       datetime64[ns]
isStart                              bool
isEnd                                bool
Aurora                           category
Bataan                           category
                                ...      
Davao del Norte Q Code              uint8
Davao del Sur Q Code                uint8
Davao Occidental Q Code             uint8
Davao Oriental Q Code               uint8
Davao City Q Code                   uint8
Length: 96, dtype: object


In [47]:
# Add Mode of Quarantine Level of all provinces
provinces = all_provinces_array()
df_regional_quarantine_timeline['mode'] = df_regional_quarantine_timeline[provinces].mode(axis=1)[0]
df_regional_quarantine_timeline['mode'] = df_regional_quarantine_timeline['mode'].astype(quarantine_cat_type)
df_regional_quarantine_timeline['mode Q Code'] = df_regional_quarantine_timeline['mode'].cat.codes

# Add Mode of Quarantine Level of each region

provinces = province_to_locations_array("III").tolist()
df_regional_quarantine_timeline['region_iii_mode'] = df_regional_quarantine_timeline[provinces].mode(axis=1)[0]
df_regional_quarantine_timeline['region_iii_mode'] = df_regional_quarantine_timeline['region_iii_mode'].astype(quarantine_cat_type)
df_regional_quarantine_timeline['region_iii_mode Q Code'] = df_regional_quarantine_timeline['region_iii_mode'].cat.codes

provinces = province_to_locations_array("IV-A").tolist()
df_regional_quarantine_timeline['region_iva_mode'] = df_regional_quarantine_timeline[provinces].mode(axis=1)[0]
df_regional_quarantine_timeline['region_iva_mode'] = df_regional_quarantine_timeline['region_iva_mode'].astype(quarantine_cat_type)
df_regional_quarantine_timeline['region_iva_mode Q Code'] = df_regional_quarantine_timeline['region_iva_mode'].cat.codes

provinces = province_to_locations_array("NCR").tolist()
df_regional_quarantine_timeline['region_ncr_mode'] = df_regional_quarantine_timeline[provinces].mode(axis=1)[0]
df_regional_quarantine_timeline['region_ncr_mode'] = df_regional_quarantine_timeline['region_ncr_mode'].astype(quarantine_cat_type)
df_regional_quarantine_timeline['region_ncr_mode Q Code'] = df_regional_quarantine_timeline['region_ncr_mode'].cat.codes

provinces = province_to_locations_array("VII").tolist()
# print(df_regional_quarantine_timeline[provinces].mode(axis=1))
df_regional_quarantine_timeline['region_vii_mode'] = df_regional_quarantine_timeline[provinces].mode(axis=1)[0]
df_regional_quarantine_timeline['region_vii_mode'] = df_regional_quarantine_timeline['region_vii_mode'].astype(quarantine_cat_type)
df_regional_quarantine_timeline['region_vii_mode Q Code'] = df_regional_quarantine_timeline['region_vii_mode'].cat.codes

provinces = province_to_locations_array("XI").tolist()
# print(provinces)
# print(df_regional_quarantine_timeline[provinces])
# print(df_regional_quarantine_timeline[provinces].mode(axis=1)[0])
df_regional_quarantine_timeline['region_xi_mode'] = df_regional_quarantine_timeline[provinces].mode(axis=1)[0]
df_regional_quarantine_timeline['region_xi_mode'] = df_regional_quarantine_timeline['region_xi_mode'].astype(quarantine_cat_type)
df_regional_quarantine_timeline['region_xi_mode Q Code'] = df_regional_quarantine_timeline['region_xi_mode'].cat.codes

In [48]:
# Output Generated CSV
df_regional_quarantine_timeline.to_csv("datasets/quarantine_measures.csv", index=False)